# Secondary Index query with Expression Selection on Primary Keys

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.1.11</version>
  </dependency>
</dependencies>

# Add required Imports

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


# Generate Test Data

In [4]:
%sh aql -c "truncate test"

In [5]:
%sh aql -c "set key_send true"

In [16]:
%sh aql -f "./aqlScripts/fruits.aql"

In [17]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Key key = new Key("test", "testset", "key1");
System.out.println("Working with record key:");
System.out.println(key);

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Initialized the client and connected to the cluster.
Working with record key:
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1
Read back the record.
Record values are:
(gen:1),(exp:409022033),(bins:(name:Jack),(age:26))


In [18]:
%sh aql -c "create index idx_age on test.testset (age) numeric"

In [25]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.Exp.Type;
import com.aerospike.client.query.RegexFlag;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,60));
QueryPolicy qp = new QueryPolicy();

//String[] keys = {"key1", "key2", "key8"};
Exp exp = Exp.val(false);  //init to false
//for (int i=0; i<keys.length; i++)
//{
   //exp = Exp.or(exp, Exp.eq(Exp.key(Type.STRING), Exp.val(keys[i]))); 
   exp = Exp.regexCompare("^.*apple.*",RegexFlag.ICASE,Exp.key(Type.STRING)); 
//}
qp.filterExp = Exp.build(exp);

// Add Expression
/*
qp.filterExp = Exp.build(Exp.or( 
  Exp.eq(Exp.key(Type.STRING), Exp.val("key1")),   //Jack
  Exp.eq(Exp.key(Type.STRING), Exp.val("key2")),   //Jill
  Exp.eq(Exp.key(Type.STRING), Exp.val("key8"))    //Sam, age 12, SI age reject.
  )
  );

*/


RecordSet rs = client.query(qp, stmt);
while (rs.next()){
  Record r = rs.getRecord();
  Key k = rs.getKey();  
  System.out.println(r);
    System.out.println(k);
}

(gen:1),(exp:409022737),(bins:(name:Sean),(age:24))
test:testset:SeanaPPlekey7:ef8ea2cf3c6f526b567acdae097ba99e6f9def2d
(gen:1),(exp:409022737),(bins:(name:Jim),(age:46))
test:testset:Jimkey4apple:eac9d3f45adb345e71842295828bdac97e7c2fb8
(gen:1),(exp:409022737),(bins:(name:Jack),(age:26))
test:testset:jacksapplekey1:acbb041d06610ece7dc1e3bea2fd0c3916cde7bb
